<a href="https://colab.research.google.com/github/aubricot/nrrd2nifti/blob/main/nrrd2nifti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert .nrrd and .seg.nrrd files from 3D Slicer to .nii.gz (nifti) files for machine learning
---   
*Last Updated 8 March 2025*  
-Runs in Python 3-   
Convert 3D slicer volume and segmentation files into nifti format for machine learning

In [ ]:
#@title Choose where to save results
import os

# Use dropdown menu on right
save = "in my Google Drive" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]

# Mount google drive to export image tagging file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Type in the path to your project wd in form field on right
wd = "/content/drive/MyDrive/slicer" # @param ["/content/drive/MyDrive/slicer"] {"allow-input":true}
print("\nWorking with data from:")
%cd $wd

# Set up folder to save .nii.gz files
ngz_folder = "nii_gzs" # @param {"type":"string","placeholder":"nii_gzs"}
ngz_fpath = wd + '/' + ngz_folder
# Set up folder to save segmentation files (labels)
labels_fpath = ngz_fpath + '/' + 'labels'

# Make ngz_folder if it doesn't already exist
if not os.path.exists(ngz_fpath):
    print("Making ngz_folder at: ", ngz_fpath)
    os.makedirs(ngz_fpath)
    print("Making labels_folder at : ", labels_fpath)
    os.makedirs(labels_fpath)
else:
    print("\nngz_folder and labels_folder already exist at: ", ngz_fpath, labels_fpath)

In [ ]:
#@title Crawl through wd to find .nrrd and _crop.nrrd files, convert them to .nii.gz, save to ngz_folder
!pip install SimpleITK
import SimpleITK as sitk

# List input and output filenames
in_vols = [] # volume files (.nrrd or _crop.nrrd)
out_vols = [] # volume files (.nii.gz)
in_segs = [] # segmentation files (.seg.nrrd)
out_segs = [] # segmentation files (_seg.nii.gz)

# Directories to search for volumes and segmentation labels
include = ['the', 'THE', 'The'] # Only use folders for theobroma cacao (thecac)
dirs = os.listdir('.')
dirs = [dir for dir in dirs if any(y in dir for y in include)]

# Make a list of directories to check if missing files
dirs_to_check = []

# Walk through chosen directories
for dir in dirs:
    files = os.listdir(dir)
    files = [f for f in files if dir in f]
    print("\n \033[92m Searching files in: {}\033[0m".format(dir))
    seg_file = [f for f in files if f.endswith("seg.nrrd")]
    vol_file = [f for f in files if "_crop" in f]
    if not vol_file:
        vol_file = [f for f in files if ("mask" not in f) and (f.endswith(".nrrd"))]

    # Warning message
    if not seg_file and vol_file:
        print("\n \033[31m Missing segmentation or volume file!\033[0m")
        print("Check files at: ", dir)
        dirs_to_check.append(dir)
        pass

    # Success message
    else:
        print("\033[92m Found segmentation and volume files!\033[0m")
        print("seg_file: ", seg_file)
        print("vol_file: ", vol_file)

        # Hacky workaround for list comprehension results
        seg_file = seg_file[0]
        vol_file = vol_file[0]

        # Segs
        img = sitk.ReadImage(dir + '/' + seg_file)
        outpath = labels_fpath + '/' + seg_file[:-9] + '_seg.nii.gz'
        print("seg_file saved to: ", outpath)
        sitk.WriteImage(img, outpath)
        in_segs.append(seg_file)
        out_segs.append(outpath)

        # Vols
        img = sitk.ReadImage(dir + '/' + vol_file)
        outpath = ngz_fpath + '/' + vol_file[:-5] + '.nii.gz'
        print("vol_file saved to: ", outpath)
        sitk.WriteImage(img, outpath)
        in_vols.append(vol_file)
        out_vols.append(outpath)

print("{} .nrrd files and {} seg.nrrd files from {} converted to .nii.gz at \n volumes: {} \n labels: {}".format(len(in_vols), len(in_segs), wd, ngz_fpath, labels_fpath))
print("{} folders were missing seg or vol files. Check on folders: {}".format(len(dirs_to_check), dirs_to_check))

# Save dirs_to_check to txt file in your wd for record of missing files
with open("dirs_to_check.txt", "w") as output:
    output.write(str(dirs_to_check))
print("Saved dirs_to_check list to: ", (wd + '/dirs_to_check.txt'))